### Run multiple experiments without the Workbench

Workflow to run multiple experiments without using the workbench. For example for as prepatory step before testing the TP identification algorithm (Figure 5 of paper).

In [1]:
from classes import *
import copy
from collections import OrderedDict
import csv
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from pathlib import Path
#import ipywidgets as widgets
import pickle

from tqdm.notebook import tqdm


from models import Rotty
from mayors import *

In [2]:
#global pars (for the development of Trust) Will be removed 
k = 0.2 
T_eq = 70
Rotty

Rotty 
-----------
allFloodProtection: 
No 3.5; Dike 4.5;  
-----------
allResidentialArea: 
Area_A Protected by:['No']
Area_B Protected by:['Dike']

Parameters : 
{'alarming_conditions': OrderedDict([(1, ['Major flood', 40]), (0, ['Small flood', 20]), (-0.2, ['Near miss', 10]), (-0.4, ['Major wake-up', 7]), (-1, ['Minor wake-up', 5]), (-10, ['Nothing happens', 0])]), 'Gumbel': OrderedDict([('mu', 2.33), ('beta', 0.234)]), 'I_experience_interp': {'xp': [0, 0.5], 'fp': [0, 1]}, 'I_social': 1}

In [3]:
select_SLR = ['1','2','3','4','5','7','9']
select_SH = ['0','1','2','3','4','124','272','312','429','451']

reset_scenarios() #Does not work well... why?

# Load and filter SLR_Scenario
allSLR_Scenario = SLR_Scenario_from_pickles(Path('SLR_projections','Transients'))
selected_SLR_Scenario = [x for x in allSLR_Scenario if x.name[-1] in select_SLR]

#Load all SurgeHeight (Realisations of extreme value distribution)
for path in Path('SurgeHeight','biased_from_five_hundred').glob('*.csv'):
    if path.stem.split('_')[-1] in select_SH:
        obj = SurgeHeight(path.stem) #Init object with name derived from filename
        obj.from_csv(path) #Fill the object with data from the csv file

# Create SurgeLevels by summing combinations of SLR_Scenario and SurgeHeights
for SLR_Scenario in selected_SLR_Scenario:
    for SH in allSurgeHeight:
        combine_SurgeLevel(SLR_Scenario,SH)
        
print(len(allSurgeLevel))

allSLR_Scenario, allSurgeHeight, and allSurgeLevel emptied
70


In [4]:
allMayors = [Reactive(),Economicus(),Sentiment(),Lawkeeper()]

In [5]:
Implementation_times = [ #small measure, large measure
    (4,6),
    #(5,7),
    #(6,9), 
    (7,10), #default
    #(8,11),
    (9,13),
    #(10,14),
    (11,16),
    #(12,17),
    #(13,19),
    (14,20)]

In [6]:
len(allSurgeLevel)*len(allMayors)*len(Implementation_times)

1400

In [8]:
from run_model import run_model01

In [9]:
#run all Majors
allExperiments = []
with tqdm(total =len(allSurgeLevel)*len(allMayors)*len(Implementation_times)) as pbar:
    for SurgeLevel in allSurgeLevel:
        #print("\n \n {} \n\n------------- \n\n".format(SurgeLevel.name))
        for Mayor in allMayors:
            for implementation_time in Implementation_times:
                experiment = run_model01(Rotty,SurgeLevel,Mayor,implementation_time,do_print=False)
                allExperiments.append(experiment)
            pbar.update(1)
print('experiments finished')


experiments finished


In [10]:
experiments=allExperiments

target = Path("temp","experiments")

today = datetime.date(datetime.now())
date = "{}_{}_{}".format(today.year,today.month,today.day)

assert target.exists()
pickle.dump(experiments, open( os.path.join(target,"experiment_selection_" + date +  ".p"), "wb") ) 